In [ ]:
! pip uninstall fsspec -qq -y
! pip install datasets -qq

In [ ]:
import re
import gc
import os
import sys
import json
import joblib
import ast
import collections
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from functools import partial
from typing import *
from tqdm.notebook import tqdm
from sklearn.utils.extmath import softmax
from sklearn import model_selection
from sklearn.metrics import classification_report, f1_score

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import transformers
from transformers import AdamW
import tokenizers
from datasets import Dataset

In [ ]:
def seed_all(seed = 42):
  """
  Fix seed for reproducibility
  """
  # python RNG
  import random
  random.seed(seed)

  # pytorch RNGs
  import torch
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

  # numpy RNG
  import numpy as np
  np.random.seed(seed)

In [ ]:
class config:
  SEED = 42
  KFOLD = 5
  TRAIN_FILE = '../input/chaii-hindi-and-tamil-question-answering/train.csv'
  SAVE_DIR = '.'
  MAX_LEN = 384
  MODEL = '../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'
  TOKENIZER = transformers.AutoTokenizer.from_pretrained(MODEL)
  EPOCHS = 5
  TRAIN_BATCH_SIZE = 4
  VALID_BATCH_SIZE = 4
  PAD_ON_RIGHT = TOKENIZER.padding_side == "right"
  DOC_STRIDE = 128
  GRAD_ACC = 4
  df = pd.read_csv(TRAIN_FILE)
  ANSWERS = {k: v for k, v in zip(df.id.values, df.answer_text.values)}
  del df

In [ ]:
class AverageMeter:
    """
    Computes and stores the average and current value
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class EarlyStopping:
    """
    Early stopping utility
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [ ]:
def prepare_train_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_token_type_ids=True,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    tokenized_examples['id'] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        tokenized_examples['id'].append(examples['id'][sample_index])
        
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
def prepare_validation_features(examples, tokenizer, pad_on_right, max_length, doc_stride):
    # ref: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_token_type_ids=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
def postprocess_qa_predictions(
    examples, tokenizer, features, raw_predictions, n_best_size=20, max_answer_length=30, squad_v2=False
):
    # ref: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    # print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples, leave=False)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
class ChaiiDataset:
    def __init__(self, data, training=True):
        self.data = data
        self.training = training

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        if self.training:
            return {
                "ids": torch.tensor(self.data[item]["input_ids"], dtype=torch.long),
                "mask": torch.tensor(self.data[item]["attention_mask"], dtype=torch.long),
                "token_type": torch.tensor(self.data[item]["token_type_ids"], dtype=torch.long),
                "start": torch.tensor(self.data[item]["start_positions"], dtype=torch.long),
                "end": torch.tensor(self.data[item]["end_positions"], dtype=torch.long),
            }
        else:
            return {
            "ids": torch.tensor(self.data[item]["input_ids"], dtype=torch.long),
            "mask": torch.tensor(self.data[item]["attention_mask"], dtype=torch.long),
            "token_type": torch.tensor(self.data[item]["token_type_ids"], dtype=torch.long),
        }

In [ ]:
def get_loss(start, start_logits, end, end_logits):
  loss_fn = nn.CrossEntropyLoss()
  start_loss = loss_fn(start_logits, start)
  end_loss = loss_fn(end_logits, end)
  loss = start_loss + end_loss
  return loss

In [ ]:
class CHAII_Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = transformers.AutoModel.from_pretrained(config.MODEL, output_hidden_states=True)
    self.layer = nn.Linear(1024, 2)
    

  def forward(self, ids, mask, token_type, start=None, end=None):
    output = self.model(input_ids = ids,
                       attention_mask = mask,
                       token_type_ids = token_type)
    
    logits = self.layer(output[0]) 
    start_logits, end_logits = logits.split(1, dim=-1)
    
    start_logits = start_logits.squeeze(-1)
    end_logits = end_logits.squeeze(-1)
    
    loss = None
    if start is not None and end is not None:
        loss = get_loss(start, start_logits, end, end_logits)    

    return loss, start_logits, end_logits

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
class Trainer:
    def __init__(self, model, train_data_loader, valid_dataset, valid_data_loader, optimizer, scheduler, device):
        self.model = model
        self.valid_dataset = valid_dataset
        self.train_data_loader = train_data_loader #pl.ParallelLoader(train_data_loader, [device]).per_device_loader(device)
        self.valid_data_loader = valid_data_loader #pl.ParallelLoader(valid_data_loader, [device]).per_device_loader(device)
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.es = EarlyStopping(patience=5, mode="max")
        self.tokenizer = config.TOKENIZER
        
    def train_eval_fn(self, fold_idx, epoch):
        self.model.train()
        losses = AverageMeter()
        para_loader = self.train_data_loader
        tk0 = tqdm(para_loader, total=len(para_loader))
      
        for bi, d in enumerate(tk0):
            ids = d['ids']
            mask = d['mask']
            token_type = d['token_type']
            start = d['start']
            end = d['end']
            
            ids = ids.to(self.device, dtype=torch.long)
            mask = mask.to(self.device, dtype=torch.long)
            token_type = token_type.to(self.device, dtype= torch.long)
            start = start.to(self.device, dtype= torch.long)
            end = end.to(self.device, dtype= torch.long)
            
            loss, start_logits, end_logits = self.model(ids, mask, token_type, start, end)
            
            loss = loss/config.GRAD_ACC
            loss.backward()
            
            if bi%config.GRAD_ACC == 0 or bi==len(self.train_data_loader) - 1:
                self.optimizer.step()
                self.model.zero_grad()
            
            if bi==len(self.train_data_loader)-1:
                valid_loss = self.eval_fn()
                print(f'Epoch : {epoch + 1} | Validation Score : {valid_loss}')
                if fold_idx is None:
                    self.es(valid_loss, self.model, model_path=os.path.join(config.SAVE_DIR, f"model.bin"))
                else:
                    self.es(valid_loss, self.model, model_path=os.path.join(config.SAVE_DIR, f"model_{fold_idx}.bin"))
                del valid_loss

            losses.update(loss.item(), ids.size(0))
            tk0.set_postfix(loss=losses.avg)
            
            del ids, token_type, mask, start, end, loss, d
            gc.collect()

        self.scheduler.step()
        del para_loader, losses, tk0
        gc.collect()
  

    def eval_fn(self):
        self.model.eval()
        
        para_loader = self.valid_data_loader
        tk0 = tqdm(para_loader, total=len(para_loader), leave=False)
        
        start, end = [], []
        
        for bi, d in enumerate(tk0):
            ids = d['ids']
            mask = d['mask']
            token_type = d['token_type']

            ids = ids.to(self.device, dtype=torch.long)
            mask = mask.to(self.device, dtype=torch.long)
            token_type = token_type.to(self.device, dtype= torch.long)
            with torch.no_grad():
                loss, start_logits, end_logits = self.model(ids, mask, token_type)       
            start_logits = start_logits.detach().cpu().numpy()
            end_logits = end_logits.detach().cpu().numpy()

            start.append(start_logits)
            end.append(end_logits)

            del ids, token_type, mask, loss, d
            gc.collect()
        
        gc.collect()
        
        start = np.vstack(start)
        end = np.vstack(end)
        
        prediction = postprocess_qa_predictions(self.valid_dataset, self.tokenizer,\
                                                        para_loader.dataset.data,\
                                                        (start, end))        
        scores = []
        for idx, pred_text in prediction.items():
            scores.append(jaccard(config.ANSWERS[idx], pred_text))
            
        del para_loader, tk0
        return np.mean(scores)
    
    def predict_fn(self):
        self.model.eval()
        
        para_loader = self.valid_data_loader
        tk0 = tqdm(para_loader, total=len(para_loader), leave=False)
        
        start, end = [], []
        
        for bi, d in enumerate(tk0):
            ids = d['ids']
            mask = d['mask']
            token_type = d['token_type']

            ids = ids.to(self.device, dtype=torch.long)
            mask = mask.to(self.device, dtype=torch.long)
            token_type = token_type.to(self.device, dtype= torch.long)
            with torch.no_grad():
                loss, start_logits, end_logits = self.model(ids, mask, token_type)       
            start_logits = start_logits.detach().cpu().numpy()
            end_logits = end_logits.detach().cpu().numpy()

            start.append(start_logits)
            end.append(end_logits)

            del ids, token_type, mask, loss, d
            gc.collect()
        
        gc.collect()
        
        start = np.vstack(start)
        end = np.vstack(end)
        
        return {'examples': self.valid_dataset,\
                'features': para_loader.dataset.data,\
                'start_logits': start,\
                'end_logits': end}

In [ ]:
def run(df_train, df_val, fold):
  train_dataset = Dataset.from_pandas(df_train)
  train_features = train_dataset.map(
    partial(
        prepare_train_features, 
        tokenizer=config.TOKENIZER,
        pad_on_right=config.PAD_ON_RIGHT, 
        max_length=config.MAX_LEN,
        doc_stride=config.DOC_STRIDE
    ),
    batched=True,
    remove_columns=train_dataset.column_names
  )
  train_feats_small = train_features.map(lambda example: example)

  val_dataset = Dataset.from_pandas(df_val)
  val_features = val_dataset.map(
    partial(
        prepare_validation_features, 
        tokenizer=config.TOKENIZER,
        pad_on_right=config.PAD_ON_RIGHT, 
        max_length=config.MAX_LEN,
        doc_stride=config.DOC_STRIDE
    ),
    batched=True,
    remove_columns=val_dataset.column_names
  )
  val_feats_small = val_features.map(lambda example: example) #, remove_columns=['example_id', 'offset_mapping'])
  
  train_data_loader = torch.utils.data.DataLoader(
        ChaiiDataset(train_feats_small),
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4,
        pin_memory=True,
    )

  valid_data_loader = torch.utils.data.DataLoader(
        ChaiiDataset(val_feats_small, training=False),
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2,
        pin_memory=True,
    )
  

  model = CHAII_Model()
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  model.to(device)

  lr = 2e-5
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
    
  trainer_obj = Trainer(model, train_data_loader, val_dataset, valid_data_loader, optimizer, scheduler, device)

  print('Starting training....')
  for epoch in range(config.EPOCHS):
    trainer_obj.train_eval_fn(fold, epoch)
    if trainer_obj.es.early_stop:
      break
  
  oof_preds = trainer_obj.predict_fn()
  joblib.dump(oof_preds, f'oof_{fold}.pkl')

  return trainer_obj.es.best_score

In [ ]:
def format_answer(answer_start, answer):
    return {'answer_start': [answer_start], 'text': [answer]}

In [ ]:
def run_k_fold(fold_id):
  '''
    Perform k-fold cross-validation
  '''
  seed_all(config.SEED)

  df_train = pd.read_csv(config.TRAIN_FILE)
  df_train['answers'] = df_train.apply(lambda x: format_answer(x['answer_start'], x['answer_text']), axis=1)
    
  train = df_train
  
  # dividing folds
  kf = model_selection.StratifiedKFold(n_splits=config.KFOLD, shuffle=False, random_state=config.SEED)
  for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.language.values)):
      train.loc[val_idx, 'kfold'] = fold

  print(f'################################################ Fold {fold_id} #################################################')
  df_train = train[train.kfold!=fold_id]
  df_val = train[train.kfold==fold_id]

  return run(df_train, df_val, fold_id)
    

In [ ]:
j0 = run_k_fold(0)

In [ ]:
! ls